In [28]:

# 导入sqlite3库
import sqlite3

# 连接到数据库
conn = sqlite3.connect('FlowerShop.db')
cursor = conn.cursor()

# 执行SQL命令来创建Flowers表
cursor.execute('''
        CREATE TABLE Flowers (
            ID INTEGER PRIMARY KEY, 
            Name TEXT NOT NULL, 
            Type TEXT NOT NULL, 
            Source TEXT NOT NULL, 
            PurchasePrice REAL, 
            SalePrice REAL,
            StockQuantity INTEGER, 
            SoldQuantity INTEGER, 
            ExpiryDate DATE,  
            Description TEXT, 
            EntryDate DATE DEFAULT CURRENT_DATE 
        );
    ''')

# 插入5种鲜花的数据
flowers = [
    ('Rose', 'Flower', 'France', 1.2, 2.5, 100, 10, '2023-12-31', 'A beautiful red rose'),
    ('Tulip', 'Flower', 'Netherlands', 0.8, 2.0, 150, 25, '2023-12-31', 'A colorful tulip'),
    ('Lily', 'Flower', 'China', 1.5, 3.0, 80, 5, '2023-12-31', 'An elegant white lily'),
    ('Daisy', 'Flower', 'USA', 0.7, 1.8, 120, 15, '2023-12-31', 'A cheerful daisy flower'),
    ('Orchid', 'Flower', 'Brazil', 2.0, 4.0, 50, 2, '2023-12-31', 'A delicate purple orchid')
]

for flower in flowers:
    cursor.execute('''
        INSERT INTO Flowers (Name, Type, Source, PurchasePrice, SalePrice, StockQuantity, SoldQuantity, ExpiryDate, Description) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
    ''', flower)

# 提交更改
conn.commit()

# 关闭数据库连接
conn.close()

OperationalError: table Flowers already exists

In [36]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain


In [38]:
db = SQLDatabase.from_uri("sqlite:///FlowerShop.db")
llm = OpenAI(temperature=0.0,verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)

result= db_chain.invoke("有多少种不同的鲜花?")
print(result)

result= db_chain.invoke("那种鲜花的销量最好?")
print(result)



> Entering new SQLDatabaseChain chain...
有多少种不同的鲜花?
SQLQuery:SELECT COUNT(DISTINCT Name) FROM Flowers
SQLResult: [(5,)]
Answer:5
> Finished chain.
{'query': '有多少种不同的鲜花?', 'result': '5'}


> Entering new SQLDatabaseChain chain...
那种鲜花的销量最好?
SQLQuery:SELECT "Name", SUM("SoldQuantity") AS "Total Sold" FROM "Flowers" GROUP BY "Name" ORDER BY "Total Sold" DESC LIMIT 1
SQLResult: [('Tulip', 25)]
Answer:Tulip
> Finished chain.
{'query': '那种鲜花的销量最好?', 'result': 'Tulip'}


In [39]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

In [45]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db,llm=llm),
    verbose = True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

#备注 数组里面两个问题的顺序 改变会导致第二个问题，回答出错。
questions= [
     "鲜花平均销售价格是多少？",
    "那种鲜花的存货数量最少？"
   
]


'''
res = agent_executor.invoke("鲜花平均销售价格是多少？")
print(res)
'''
for question in questions:
    res = agent_executor.invoke(question)
    print(res)




> Entering new SQL Agent Executor chain...
 We need to find the average sales price for flowers.
Action: sql_db_query
Action Input: SELECT AVG(price) FROM flowersError: (sqlite3.OperationalError) no such column: price
[SQL: SELECT AVG(price) FROM flowers]
(Background on this error at: https://sqlalche.me/e/20/e3q8)We need to check the table schema to see what columns are available.
Action: sql_db_schema
Action Input: flowersError: table_names {'flowers'} not found in databaseWe need to check the list of tables in the database to make sure the table exists.
Action: sql_db_list_tables
Action Input: ""FlowersWe need to check the table schema again to see what columns are available.
Action: sql_db_schema
Action Input: Flowers
CREATE TABLE "Flowers" (
	"ID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Type" TEXT NOT NULL, 
	"Source" TEXT NOT NULL, 
	"PurchasePrice" REAL, 
	"SalePrice" REAL, 
	"StockQuantity" INTEGER, 
	"SoldQuantity" INTEGER, 
	"ExpiryDate" DATE, 
	"Description" TEXT, 
	"EntryDate